In [1]:
!pip install -q git+https://github.com/Farama-Foundation/MAgent2

^C


# Init

In [1]:
# Libraries
import numpy as np
import torch
from magent2.environments import battle_v4
from dataclasses import dataclass

# Modules
from model import VdnQNet
from utils import TeamManager
from train import save_data, train, run_episode, run_model_train_test
from eval import evaluate_model

seed = 25
device = 'cuda' if torch.cuda.is_available() else 'cpu'

@dataclass
class VdnHyperparameters:
    lr: float = 0.001
    gamma: float = 0.99
    batch_size: int = 2048
    update_iter: int = 20
    buffer_limit: int = 9000
    update_target_interval: int = 20
    max_episodes: int = 500
    max_epsilon: float = 0.9
    min_epsilon: float = 0.1
    episode_min_epsilon: int = 200
    test_episodes: int = 10
    warm_up_steps: int = 3000
    chunk_size: int = 1
    recurrent: bool = False

# Training

In [2]:
# Training
save_name = 'vdn_blue'
loaded_model = None

# Hyperparameters
hp = VdnHyperparameters(
    lr=0.0005,
    gamma=0.99,
    batch_size=512,
    buffer_limit=9000,
    max_episodes=400,
    max_epsilon=0.9,
    min_epsilon=0.1,
    episode_min_epsilon=200,
    test_episodes=1,
    warm_up_steps=3000,
    update_iter=20,
    chunk_size=1,
    update_target_interval=20,
    recurrent=True
)
print(hp)

# Create env
env = battle_v4.parallel_env(map_size=45)
test_env = battle_v4.parallel_env(map_size=45)

env.reset(seed=seed)
test_env.reset(seed=seed)
team_manager = TeamManager(env.agents)

# Create model
q = VdnQNet(team_manager.get_my_agents(), env.observation_spaces, env.action_spaces).to(device)
q_target = VdnQNet(team_manager.get_my_agents(), env.observation_spaces, env.action_spaces).to(device)

# Run train
train_scores, test_scores, losses = run_model_train_test(
    env, test_env, q, q_target,
    save_name, team_manager, hp, train, run_episode
)

# Save data
save_data(np.array(train_scores), f'{save_name}-train_scores')
save_data(np.array(test_scores), f'{save_name}-test_scores')
save_data(np.array(losses), f'{save_name}-losses')

VdnHyperparameters(lr=0.0005, gamma=0.99, batch_size=512, update_iter=20, buffer_limit=9000, update_target_interval=20, max_episodes=400, max_epsilon=0.9, min_epsilon=0.1, episode_min_epsilon=200, test_episodes=1, warm_up_steps=3000, chunk_size=1, recurrent=True)
My team:  blue
Episodes 1 / 400
